<a href="https://colab.research.google.com/github/chetannitk/external_assignments/blob/master/sarcasm_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sarcasm Text classification with TensorFlow Hub

In [0]:
import numpy as np
import pandas as pd
import tensorflow as tf

!pip install tensorflow-hub
!pip install tfds-nightly
import tensorflow_hub as hub
import tensorflow_datasets as tfds

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.experimental.list_physical_devices("GPU") else "NOT AVAILABLE")

## Upload the Train and Test dataset



In [0]:
df_train  = pd.read_csv("https://raw.githubusercontent.com/chetannitk/external_assignments/master/datasets/Train_sarcasm.csv")
df_test  = pd.read_csv("https://raw.githubusercontent.com/chetannitk/external_assignments/master/datasets/Test_sarcasm.csv")

In [0]:
df_train.columns

In [0]:
df_train.label.value_counts()

In [0]:
df_train.info()

In [0]:
df_test.info()

## Explore the data 



In [0]:
df_train['d_text'] = df_train['parent_comment'] + ". " + df_train['comment']
df_test['d_text'] = df_test['parent_comment'] + ". " + df_test['comment']

In [0]:
train_examples_batch = df_train['d_text']
train_labels_batch = df_train['label']
train_examples_batch

In [0]:
train_labels_batch

## Build the model


In [0]:
#embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
embedding = "https://tfhub.dev/google/nnlm-en-dim50/2"

hub_layer = hub.KerasLayer(embedding, input_shape=[], 
                           dtype=tf.string, trainable=True)
hub_layer(train_examples_batch[:3])

Let's now build the full model:

In [0]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(128, activation='sigmoid'))
model.add(tf.keras.layers.Dense(1))

model.summary()

In [0]:
model.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), metrics=['accuracy'])

## Train the model


In [0]:
history = model.fit(train_examples_batch, train_labels_batch, epochs=20, validation_split=0.2, batch_size=32,
                    verbose=1)

## Evaluate the model

And let's see how the model performs. Two values will be returned. Loss (a number which represents our error, lower values are better), and accuracy.

In [0]:
results = model.predict_classes(df_test['d_text'].tolist())


In [0]:
results